In [ ]:
import numpy as np
import tensorflow as tf
import pickle
import os
import yaml
import re

In [ ]:
dir_path = r'C:\Users\ludwi\Desktop\JuFo_DeepLearning\ChatbotData'
files = os.listdir(dir_path + os.sep)

fragen = list()
antworten = list()

for path in files:
    f = open( dir_path + os.sep + path , 'rb')
    docs = yaml.safe_load(f)
    chat = docs['conversations']
    for con in chat:
        if len( con ) > 2 :
            fragen.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            antworten.append( ans )
        elif len( con )> 1:
            fragen.append(con[0])
            antworten.append(con[1])

antworten_mit_tags = list()

for i in range( len( antworten ) ):
    if type( antworten[i] ) == str:
        antworten_mit_tags.append( antworten[i] )
    else:
        fragen.pop( i )

antworten = list()

for i in range( len( antworten_mit_tags ) ) :
    antworten.append( '<START> ' + antworten_mit_tags[i] + ' <END>' )

tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( fragen + antworten )
VOCAB_SIZE = len( tokenizer.word_index )+1
#print( 'VOCAB SIZE : ',VOCAB_SIZE ))


In [ ]:
# encoder_input_data
tokenized_fragen = tokenizer.texts_to_sequences( fragen )
maxlen_fragen = max( [ len(x) for x in tokenized_fragen ] )
padded_fragen = tf.keras.preprocessing.sequence.pad_sequences( tokenized_fragen , maxlen=maxlen_fragen , padding='post' )
encoder_input_data = np.array( padded_fragen )

# decoder_input_data
tokenized_antworten = tokenizer.texts_to_sequences( antworten )
maxlen_antworten = max( [ len(x) for x in tokenized_antworten ] )
padded_antworten = tf.keras.preprocessing.sequence.pad_sequences( tokenized_antworten , maxlen=maxlen_antworten , padding='post' )
decoder_input_data = np.array( padded_antworten )

# decoder_output_data
tokenized_antworten = tokenizer.texts_to_sequences( antworten )

for i in range(len(tokenized_antworten)) :
    tokenized_antworten[i] = tokenized_antworten[i][1:]
padded_antworten = tf.keras.preprocessing.sequence.pad_sequences( tokenized_antworten , maxlen=maxlen_antworten , padding='post' )
onehot_antworten = tf.keras.utils.to_categorical( padded_antworten , VOCAB_SIZE )
decoder_output_data = np.array( onehot_antworten )

In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_fragen , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_antworten ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary(200)


In [ ]:
logdir=str(r'C:\Users\ludwi\Desktop\JuFo_DeepLearning\TensorBoard' + r'\AutoEncoder-Chatbot')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:

model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150, callbacks=[tensorboard_callback] )


In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


In [ ]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return tf.keras.preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_fragen , padding='post')

crash bei unbekanntem wort

In [ ]:
enc_model , dec_model = make_inference_models()

while True:
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']

    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' ' + word
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_antworten:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )
